In [1]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [ ]:
default_size=224


model =  Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(default_size, default_size, 3), activation='relu', border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

In [ ]:
weights_path = '../weights_model/vgg16_weights_tf_dim_ordering_tf_kernels.h5'

f = h5py.File(weights_path)
for k, layer_name in enumerate(f.attrs[u'layer_names']):
    if k == len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f[layer_name]
    weights = [g[p] for p in g.keys()]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

In [2]:
train_data_dir = '../train_data/padded_letters/'
nb_train_samples = 1555
#60 examples per class
#except 't' only has 5 examples

In [3]:
default_size = 244

In [4]:


datagen = ImageDataGenerator(rescale=1., 
                             featurewise_center=True)

datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(default_size, default_size),
            batch_size=32,
            class_mode="categorical",
            shuffle=False
            )

Found 1555 images belonging to 26 classes.


In [ ]:
import pandas as pd

cut = pd.DataFrame(generator.filenames)
cut["label"] = cut.iloc[:,0].str[0]
cut.head()

In [5]:
from keras.applications import VGG16

model = VGG16(include_top=False)

In [8]:
bottleneck_features_train = model.predict_generator(generator, nb_train_samples)

KeyboardInterrupt: 

In [ ]:
np.save(open('vgg_bottleneck.npy', 'w'), bottleneck_features_train)

In [ ]:
np.save(open('vgg_bottleneck.npy', 'w'), bottleneck_features_train)
labels = list(generator.classes)
np.save(open('vgg_bottle_labels.npy', 'w'), labels)

In [ ]:
data = np.load(open('../weights_model/bottleneck_features_train.npy'))

In [ ]:
data.shape

In [ ]:
labels = list(generator.classes)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=123)
sss.get_n_splits(data, labels)

In [ ]:
X=data.copy()
y= np.copy(labels)

for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
from keras.utils.np_utils import to_categorical

train_labels = to_categorical(y_train, nb_classes=26)
test_labels = to_categorical(y_test, nb_classes=26)

In [ ]:
print train_labels.shape
print test_labels.shape

In [ ]:
nb_epoch = 30

top_model = Sequential()
top_model.add(Flatten(input_shape=X_test.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(26, activation='softmax'))

top_model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

history = top_model.fit(X_train, train_labels,
          nb_epoch=nb_epoch, 
              batch_size=32,
          validation_data=(X_test, test_labels))

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=X_test.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(26, activation='softmax'))

In [ ]:
top_model_weights_path = 'bottleneck_fc_model.h5'

In [ ]:
top_model.save_weights(top_model_weights_path)

In [ ]:
###############

In [ ]:
import matplotlib 
matplotlib.use('Agg') 
% matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def plot_logs(history):
    evaluation_cost = history.history['val_loss']
    evaluation_accuracy = history.history['val_acc']
    training_cost = history.history['loss']
    training_accuracy = history.history['acc']
    f, (ax1, ax2) = plt.subplots(1, 2)
    f.set_figwidth(10)
    ax1.plot(evaluation_cost,label= 'test')
    ax1.plot(training_cost, label='train')
    ax1.set_title('Cost')
    ax1.legend()
    ax2.plot(evaluation_accuracy, label='test')
    ax2.plot(training_accuracy, label='train')
    ax2.set_title('Accuracy')
    ax2.legend(loc='lower right')

In [ ]:
plot_logs(history)

In [ ]:
##################

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=(7,7,512)))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(26, activation='softmax'))

top_model.load_weights("../weights_model/bottleneck_fc_model.h5")

In [ ]:
import os
import requests
from io import StringIO, BytesIO
from PIL import Image

## helper functions
def load_image_from_url(image_url):
    request_respone = requests.get(image_url).content
    image = Image.open(BytesIO(request_respone))
    return image

def preprocess_for_vgg(image):
    image = image.resize([224,224])
    x = np.array(image, dtype=float)
    x_fake_batch = x.reshape(1,*x.shape)
    x = x_fake_batch
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    return x

In [ ]:
url = 'https://previews.123rf.com/images/belchonock/belchonock1309/belchonock130901540/22021906-Finger-Spelling-the-Alphabet-in-American-Sign-Language-ASL-Letter-U-Stock-Photo.jpg'
numpy_array = preprocess_for_vgg(load_image_from_url(url))
numpy_array.shape

In [ ]:
image = Image.open("/Users/Belal/Desktop/hello.jpg")
numpy_array = preprocess_for_vgg(image)
numpy_array.shape

In [ ]:
step_1.shape

In [ ]:
step_1 = model.predict(numpy_array)

predictions = top_model.predict(step_1, batch_size=step_1.shape[0])[0]
top_prediction = np.argmax(predictions)

predict_labels = {}
for k in range(1,27):
    predict_labels[k]=string.ascii_uppercase[k-1]

predict_labels.get(top_prediction)